# Earlier version, was creating artifacts, but now fixed

In [1]:
import cv2
import numpy as np

# Load images
foreground = cv2.imread('car.png', cv2.IMREAD_UNCHANGED)
background = cv2.imread('street.jpg')

if foreground.shape[2] == 4:
    # 4 channels imply alpha channel present
    alpha_channel = foreground[:, :, 3]  # Get the alpha channel
    mask = alpha_channel
    foreground = foreground[:, :, :3]    # Remove the alpha channel for blending
else:
    # if no alpha channel then create one
    mask = cv2.cvtColor(foreground, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

# Resizing foreground if it's too large for the background
bg_height, bg_width = background.shape[:2]
fg_height, fg_width = foreground.shape[:2]

# Ensure the foreground fits within the background
if fg_width > bg_width or fg_height > bg_height:
    scale_factor = min(bg_width / fg_width, bg_height / fg_height)
    foreground = cv2.resize(foreground, (int(fg_width * scale_factor), int(fg_height * scale_factor)))
    mask = cv2.resize(mask, (int(fg_width * scale_factor), int(fg_height * scale_factor)))

# x-axis position in the middle of the background
center_x = bg_width // 2
# y-axis position in the lower part of the background
center_y = int(bg_height * 0.75)  
center = (center_x, center_y)

# Poisson blending
blended_image = cv2.seamlessClone(foreground, background, mask, center, cv2.NORMAL_CLONE)

cv2.imshow('Blended Image', blended_image)
cv2.imwrite('blended_car_on_street.jpg', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()